## Libraries

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stat

from math import sqrt
from mlgear.utils import show, display_columns
from surveyweights import normalize_weights, run_weighting_iteration


def margin_of_error(n=None, sd=None, p=None, type='proportion', interval_size=0.95):
    z_lookup = {0.8: 1.28, 0.85: 1.44, 0.9: 1.65, 0.95: 1.96, 0.99: 2.58}
    if interval_size not in z_lookup.keys():
        raise ValueError('{} not a valid `interval_size` - must be {}'.format(interval_size,
                                                                              ', '.join(list(z_lookup.keys()))))
    if type == 'proportion':
        se = sqrt(p * (1 - p)) / sqrt(n)
    elif type == 'continuous':
        se = sd / sqrt(n)
    else:
        raise ValueError('{} not a valid `type` - must be proportion or continuous')
    
    z = z_lookup[interval_size]
    return se * z


def print_pct(pct, digits=0):
    pct = pct * 100
    pct = np.round(pct, digits)
    if pct >= 100:
        if digits == 0:
            val = '>99.0%'
        else:
            val = '>99.'
            for d in range(digits - 1):
                val += '9'
            val += '9%'
    elif pct <= 0:
        if digits == 0:
            val = '<0.1%'
        else:
            val = '<0.'
            for d in range(digits - 1):
                val += '0'
            val += '1%'
    else:
        val = '{}%'.format(pct)
    return val


def calc_result(for_vote, against_vote, n, state=None, interval=0.8):
    GENERAL_POLLING_ERROR = 4.0
    ACQUIESENCE_BIAS = -5.0
    N_SIMS = 1000000
    
    margin_needed_for_victory = 0.5
    if state == 'Florida':
        margin_needed_for_victory = 0.6
    elif state == 'Colorado':
        margin_needed_for_victory = 0.55
    
    for_moe = margin_of_error(n=n, p=for_vote/100, interval_size=interval)
    against_moe = margin_of_error(n=n, p=against_vote/100, interval_size=interval)
    undecided = 100 - for_vote - against_vote
    mean = for_vote + undecided * 0.25 + ACQUIESENCE_BIAS
    raw_moe = for_moe * 100 + against_moe * 100
    
    allocate_undecided = undecided * 0.4
    margin = raw_moe + allocate_undecided + GENERAL_POLLING_ERROR
    
    cdf_value = 0.5 + 0.5 * interval
    normed_sigma = stat.norm.ppf(cdf_value)
    sigma = margin / 100 / normed_sigma
    
    sims = np.random.normal(mean / 100, sigma, N_SIMS)
    chance_pass = np.sum([s > margin_needed_for_victory for s in sims]) / N_SIMS
    low, high = np.percentile(sims, [20, 80]) * 100
    
    return {'mean': mean, 'high': high, 'low': low, 'n': n,
            'raw_moe': raw_moe, 'margin': margin, 'sigma': sigma, 'chance_pass': chance_pass}


def print_result(mean, high, low, n, raw_moe, margin, sigma, chance_pass):
    mean = np.round(mean, 1)
    first = np.round(high, 1)
    second = np.round(low, 1)
    sigma = np.round(sigma * 100, 1)
    raw_moe = np.round(raw_moe, 1)
    margin = np.round(margin, 1)
    chance_pass = print_pct(chance_pass, 1)
    if second < first:
        _ = first
        first = second
        second = _
    if second > 100:
        second = 100
    if first < 0:
        first = 0
    print(('Result {} (80% CI: {} to {}) (N={}) (raw_moe={}pts, margin={}pts, '
           'sigma={}pts) ({} likely to pass)').format(mean,
                                                      first,
                                                      second,
                                                      n,
                                                      raw_moe,
                                                      margin,
                                                      sigma,
                                                      chance_pass))
    print(('{} (80% CI: {} to {}) ({})').format(mean,
                                                first,
                                                second,
                                                chance_pass))
    print('-')

## Load Processed Data

In [2]:
survey = pd.read_csv('responses_processed_national_weighted.csv').fillna('Not presented')

/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,78,79,80,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Other Propositions

In [3]:
measures = ['vote_measure_primate_rights', 'vote_measure_cage_free', 'vote_measure_psilocybin',
            'vote_measure_rank_choice', 'vote_measure_no_labeling', 'vote_measure_marijuana',
            'vote_measure_alt_protein_funding', 'vote_measure_utah_hunt', 'vote_measure_min_wage']

options = ['Vote for / Support', 'Vote against / Oppose', 'Don’t know / Undecided']


for measure in measures:
    print('## {} ##'.format(measure))    
    survey_ = survey.copy()
    survey_ = survey_.loc[survey_[measure].isin(options)].copy()
    survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
    survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
    survey_['lv_weight_2020'] = normalize_weights(survey_['lv_weight_2020'])

    print('# LV Weighted')
    lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
    votes = survey_[measure].value_counts(normalize=True) * survey_.groupby(measure)['lv_weight'].mean() * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print('-')
    print('Raw N: {}'.format(len(survey_)))
    print_result(**calc_result(for_vote=votes['Vote for / Support'],
                               against_vote=votes['Vote against / Oppose'],
                               n=lv_weighted_n))
    
    print('# LV Weighted (Alt, Post-Hoc)')
    lv_weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
    votes = survey_[measure].value_counts(normalize=True) * survey_.groupby(measure)['lv_weight_alt'].mean() * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print_result(**calc_result(for_vote=votes['Vote for / Support'],
                               against_vote=votes['Vote against / Oppose'],
                               n=lv_weighted_n))
    
    print('# LV Weighted (2020 National, Post-Hoc)')
    lv_weighted_n = int(np.round(survey_['lv_weight_2020'].apply(lambda w: 1 if w > 1 else w).sum()))
    votes = survey_[measure].value_counts(normalize=True) * survey_.groupby(measure)['lv_weight_2020'].mean() * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print_result(**calc_result(for_vote=votes['Vote for / Support'],
                               against_vote=votes['Vote against / Oppose'],
                               n=lv_weighted_n))

    print('-')
    print('-')
    print('-')

## vote_measure_primate_rights ##
# LV Weighted
Vote for / Support        37.760546
Vote against / Oppose     40.087809
Don’t know / Undecided    22.151645
dtype: float64
-
Raw N: 639
Result 38.3 (80% CI: 24.7 to 51.8) (N=257) (raw_moe=7.8pts, margin=20.6pts, sigma=16.1pts) (23.4% likely to pass)
38.3 (80% CI: 24.7 to 51.8) (23.4%)
-
# LV Weighted (Alt, Post-Hoc)
Vote for / Support        33.133366
Vote against / Oppose     40.798090
Don’t know / Undecided    26.068544
dtype: float64
Result 34.7 (80% CI: 20.0 to 49.3) (N=244) (raw_moe=7.9pts, margin=22.3pts, sigma=17.4pts) (18.9% likely to pass)
34.7 (80% CI: 20.0 to 49.3) (18.9%)
-
# LV Weighted (2020 National, Post-Hoc)
Vote for / Support        37.300060
Vote against / Oppose     40.826138
Don’t know / Undecided    21.873801
dtype: float64
Result 37.8 (80% CI: 24.3 to 51.3) (N=253) (raw_moe=7.8pts, margin=20.6pts, sigma=16.1pts) (22.4% likely to pass)
37.8 (80% CI: 24.3 to 51.3) (22.4%)
-
-
-
-
## vote_measure_cage_free ##
# LV Weig

Result 68.3 (80% CI: 59.6 to 77.1) (N=279) (raw_moe=6.6pts, margin=13.3pts, sigma=10.4pts) (96.1% likely to pass)
68.3 (80% CI: 59.6 to 77.1) (96.1%)
-
-
-
-


In [4]:
POTUS_CENSUS = {'Alabama': {'Hillary Clinton': 0.3436, 'Donald Trump': 0.6208},
                'Alaska': {'Hillary Clinton': 0.3655, 'Donald Trump': 0.5128},
                'Arizona': {'Hillary Clinton': 0.4513, 'Donald Trump': 0.4867},
                'Arkansas': {'Hillary Clinton': 0.3365, 'Donald Trump': 0.6057},
                'California': {'Hillary Clinton': 0.6173, 'Donald Trump': 0.3162},
                'Colorado': {'Hillary Clinton': 0.4816, 'Donald Trump': 0.4325},
                'Connecticut': {'Hillary Clinton': 0.5457, 'Donald Trump': 0.4093},
                'Delaware': {'Hillary Clinton': 0.531, 'Donald Trump': 0.417},
                'Washington DC': {'Hillary Clinton': 0.905, 'Donald Trump': 0.016},
                'Florida': {'Hillary Clinton': 0.478, 'Donald Trump': 0.490},
                'Georgia': {'Hillary Clinton': 0.456, 'Donald Trump': 0.508},
                'Hawaii': {'Hillary Clinton': 0.622, 'Donald Trump': 0.300},
                'Idaho': {'Hillary Clinton': 0.275, 'Donald Trump': 0.593},
                'Illinois': {'Hillary Clinton': 0.558, 'Donald Trump': 0.379},
                'Indiana': {'Hillary Clinton': 0.379, 'Donald Trump': 0.511},
                'Iowa': {'Hillary Clinton': 0.417, 'Donald Trump': 0.512},
                'Kansas': {'Hillary Clinton': 0.361, 'Donald Trump': 0.567},
                'Kentucky': {'Hillary Clinton': 0.327, 'Donald Trump': 0.625},
                'Louisiana': {'Hillary Clinton': 0.385, 'Donald Trump': 0.581},
                'Maine': {'Hillary Clinton': 0.478, 'Donald Trump': 0.449},
                'Maryland': {'Hillary Clinton': 0.603, 'Donald Trump': 0.339},
                'Massachusetts': {'Hillary Clinton': 0.600, 'Donald Trump': 0.328},
                'Michigan': {'Hillary Clinton': 0.473, 'Donald Trump': 0.475},
                'Minnesota': {'Hillary Clinton': 0.464, 'Donald Trump': 0.449},
                'Mississippi': {'Hillary Clinton': 0.401, 'Donald Trump': 0.579},
                'Missouri': {'Hillary Clinton': 0.401, 'Donald Trump': 0.579},
                'Montana': {'Hillary Clinton': 0.381, 'Donald Trump': 0.562},
                'Nebraska': {'Hillary Clinton': 0.337, 'Donald Trump': 0.588},
                'Nevada': {'Hillary Clinton': 0.479, 'Donald Trump': 0.455},
                'New Hampshire': {'Hillary Clinton': 0.470, 'Donald Trump': 0.466},
                'New Jersey': {'Hillary Clinton': 0.555, 'Donald Trump': 0.414},
                'New Mexico': {'Hillary Clinton': 0.483, 'Donald Trump': 0.404},
                'New York': {'Hillary Clinton': 0.590, 'Donald Trump': 0.365},
                'North Carolina': {'Hillary Clinton': 0.462, 'Donald Trump': 0.498},
                'North Dakota': {'Hillary Clinton': 0.272, 'Donald Trump': 0.630},
                'Ohio': {'Hillary Clinton': 0.436, 'Donald Trump': 0.517},
                'Oklahoma': {'Hillary Clinton': 0.289, 'Donald Trump': 0.653},
                'Oregon': {'Hillary Clinton': 0.501, 'Donald Trump': 0.391},
                'Pennsylvania': {'Hillary Clinton': 0.475, 'Donald Trump': 0.481},
                'Rhode Island': {'Hillary Clinton': 0.544, 'Donald Trump': 0.389},
                'South Carolina': {'Hillary Clinton': 0.407, 'Donald Trump': 0.549},
                'South Dakota': {'Hillary Clinton': 0.317, 'Donald Trump': 0.615},
                'Tennessee': {'Hillary Clinton': 0.347, 'Donald Trump': 0.607},
                'Texas': {'Hillary Clinton': 0.432, 'Donald Trump': 0.522},
                'Utah': {'Hillary Clinton': 0.275, 'Donald Trump': 0.454},
                'Vermont': {'Hillary Clinton': 0.567, 'Donald Trump': 0.303},
                'Virginia': {'Hillary Clinton': 0.497, 'Donald Trump': 0.444},
                'Washington': {'Hillary Clinton': 0.525, 'Donald Trump': 0.368},
                'West Virginia': {'Hillary Clinton': 0.264, 'Donald Trump': 0.685},
                'Wisconsin': {'Hillary Clinton': 0.465, 'Donald Trump': 0.472},
                'Wyoming': {'Hillary Clinton': 0.216, 'Donald Trump': 0.674 }}

for measure in measures:
    for state, _ in POTUS_CENSUS.items():
        print('## {} in {} ##'.format(measure, state))

        state_survey = survey.copy()
        potus_census = {'vote2016': POTUS_CENSUS[state].copy()}
        potus_census['vote2016']['Other'] = 1 - potus_census['vote2016']['Hillary Clinton'] - potus_census['vote2016']['Donald Trump']
        output = run_weighting_iteration(state_survey, census=potus_census, weigh_on=['vote2016'], verbose=0)
        potus_weights = output['weights']['vote2016']
        potus_weights = state_survey['vote2016'].astype(str).replace(potus_weights)
        state_survey['weight'] = normalize_weights(state_survey['weight'] * potus_weights)
        state_survey['lv_weight'] = normalize_weights(state_survey['weight'] * state_survey['lv_index'])
        state_survey['lv_weight_alt'] = state_survey['lv_weight']
        state_survey.loc[(~state_survey['voted2016']) & (state_survey['vote_trump_biden'] == 'Donald Trump, the Republican'), 'lv_weight_alt'] *= 1.662
        state_survey['lv_weight_alt'] = normalize_weights(state_survey['lv_weight_alt'])
        state_survey_ = state_survey.loc[state_survey[measure].isin(options)].copy()
        state_survey_['lv_weight_alt'] = normalize_weights(state_survey_['lv_weight_alt'])
        lv_weighted_n = int(np.round(state_survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
        votes = state_survey_[measure].value_counts(normalize=True) * state_survey_.groupby(measure)['lv_weight_alt'].mean() * 100
        votes = votes[options] * (100 / votes[options].sum())
        print(votes)
        print('Raw N: {}'.format(len(state_survey_)))
        print_result(**calc_result(for_vote=votes['Vote for / Support'],
                                   against_vote=votes['Vote against / Oppose'],
                                   state=state,
                                   n=lv_weighted_n))
        print('-')

## vote_measure_primate_rights in Alabama ##
Vote for / Support        37.664765
Vote against / Oppose     38.346939
Don’t know / Undecided    23.988296
dtype: float64
Raw N: 639
Result 38.7 (80% CI: 24.5 to 52.8) (N=242) (raw_moe=8.0pts, margin=21.6pts, sigma=16.8pts) (25.1% likely to pass)
38.7 (80% CI: 24.5 to 52.8) (25.1%)
-
-
## vote_measure_primate_rights in Alaska ##
Vote for / Support        33.022374
Vote against / Oppose     43.731299
Don’t know / Undecided    23.246327
dtype: float64
Raw N: 639
Result 33.8 (80% CI: 19.9 to 47.8) (N=240) (raw_moe=8.0pts, margin=21.3pts, sigma=16.6pts) (16.5% likely to pass)
33.8 (80% CI: 19.9 to 47.8) (16.5%)
-
-
## vote_measure_primate_rights in Arizona ##
Vote for / Support        35.65712
Vote against / Oppose     39.27008
Don’t know / Undecided    25.07280
dtype: float64
Raw N: 639
Result 36.9 (80% CI: 22.6 to 51.3) (N=250) (raw_moe=7.8pts, margin=21.9pts, sigma=17.1pts) (22.2% likely to pass)
36.9 (80% CI: 22.6 to 51.3) (22.2%)
-
-
## vo

Result 39.5 (80% CI: 25.2 to 53.9) (N=247) (raw_moe=7.9pts, margin=21.9pts, sigma=17.1pts) (27.0% likely to pass)
39.5 (80% CI: 25.2 to 53.9) (27.0%)
-
-
## vote_measure_primate_rights in Montana ##
Vote for / Support        36.277961
Vote against / Oppose     39.511500
Don’t know / Undecided    24.210539
dtype: float64
Raw N: 639
Result 37.3 (80% CI: 23.2 to 51.5) (N=245) (raw_moe=7.9pts, margin=21.6pts, sigma=16.9pts) (22.6% likely to pass)
37.3 (80% CI: 23.2 to 51.5) (22.6%)
-
-
## vote_measure_primate_rights in Nebraska ##
Vote for / Support        35.521129
Vote against / Oppose     41.058748
Don’t know / Undecided    23.420123
dtype: float64
Raw N: 639
Result 36.4 (80% CI: 22.4 to 50.4) (N=241) (raw_moe=8.0pts, margin=21.4pts, sigma=16.7pts) (20.7% likely to pass)
36.4 (80% CI: 22.4 to 50.4) (20.7%)
-
-
## vote_measure_primate_rights in Nevada ##
Vote for / Support        35.301971
Vote against / Oppose     39.313148
Don’t know / Undecided    25.384881
dtype: float64
Raw N: 639
R

Result 34.7 (80% CI: 21.0 to 48.4) (N=228) (raw_moe=8.2pts, margin=20.9pts, sigma=16.3pts) (17.4% likely to pass)
34.7 (80% CI: 21.0 to 48.4) (17.4%)
-
-
## vote_measure_cage_free in Alabama ##
Vote for / Support        75.686357
Vote against / Oppose     16.130567
Don’t know / Undecided     8.183076
dtype: float64
Raw N: 833
Result 72.7 (80% CI: 64.2 to 81.2) (N=324) (raw_moe=5.7pts, margin=12.9pts, sigma=10.1pts) (98.8% likely to pass)
72.7 (80% CI: 64.2 to 81.2) (98.8%)
-
-
## vote_measure_cage_free in Alaska ##
Vote for / Support        77.096630
Vote against / Oppose     14.821585
Don’t know / Undecided     8.081785
dtype: float64
Raw N: 833
Result 74.1 (80% CI: 65.8 to 82.4) (N=331) (raw_moe=5.5pts, margin=12.7pts, sigma=9.9pts) (99.3% likely to pass)
74.1 (80% CI: 65.8 to 82.4) (99.3%)
-
-
## vote_measure_cage_free in Arizona ##
Vote for / Support        76.388300
Vote against / Oppose     15.464327
Don’t know / Undecided     8.147373
dtype: float64
Raw N: 833
Result 73.4 (80% C

Result 72.6 (80% CI: 64.1 to 81.1) (N=329) (raw_moe=5.6pts, margin=12.9pts, sigma=10.1pts) (98.8% likely to pass)
72.6 (80% CI: 64.1 to 81.1) (98.8%)
-
-
## vote_measure_cage_free in Missouri ##
Vote for / Support        75.583389
Vote against / Oppose     16.217185
Don’t know / Undecided     8.199426
dtype: float64
Raw N: 833
Result 72.6 (80% CI: 64.1 to 81.1) (N=329) (raw_moe=5.6pts, margin=12.9pts, sigma=10.1pts) (98.8% likely to pass)
72.6 (80% CI: 64.1 to 81.1) (98.8%)
-
-
## vote_measure_cage_free in Montana ##
Vote for / Support        76.123369
Vote against / Oppose     15.720322
Don’t know / Undecided     8.156309
dtype: float64
Raw N: 833
Result 73.2 (80% CI: 64.7 to 81.6) (N=331) (raw_moe=5.6pts, margin=12.8pts, sigma=10.0pts) (99.0% likely to pass)
73.2 (80% CI: 64.7 to 81.6) (99.0%)
-
-
## vote_measure_cage_free in Nebraska ##
Vote for / Support        76.292736
Vote against / Oppose     15.570123
Don’t know / Undecided     8.137141
dtype: float64
Raw N: 833
Result 73.3 (8

Result 73.5 (80% CI: 65.1 to 81.8) (N=340) (raw_moe=5.5pts, margin=12.7pts, sigma=9.9pts) (99.1% likely to pass)
73.5 (80% CI: 65.1 to 81.8) (99.1%)
-
-
## vote_measure_cage_free in Wyoming ##
Vote for / Support        76.521991
Vote against / Oppose     15.375525
Don’t know / Undecided     8.102484
dtype: float64
Raw N: 833
Result 73.5 (80% CI: 65.0 to 82.0) (N=308) (raw_moe=5.7pts, margin=13.0pts, sigma=10.1pts) (99.0% likely to pass)
73.5 (80% CI: 65.0 to 82.0) (99.0%)
-
-
## vote_measure_psilocybin in Alabama ##
Vote for / Support        44.803564
Vote against / Oppose     33.492646
Don’t know / Undecided    21.703790
dtype: float64
Raw N: 797
Result 45.2 (80% CI: 32.4 to 58.1) (N=326) (raw_moe=6.9pts, margin=19.6pts, sigma=15.3pts) (37.7% likely to pass)
45.2 (80% CI: 32.4 to 58.1) (37.7%)
-
-
## vote_measure_psilocybin in Alaska ##
Vote for / Support        47.169344
Vote against / Oppose     31.267512
Don’t know / Undecided    21.563144
dtype: float64
Raw N: 797
Result 47.6 (80%

...

Result 49.0 (80% CI: 39.0 to 59.0) (N=303) (raw_moe=7.3pts, margin=15.2pts, sigma=11.9pts) (46.7% likely to pass)
49.0 (80% CI: 39.0 to 59.0) (46.7%)
-
-
## vote_measure_alt_protein_funding in Vermont ##
Vote for / Support        59.297453
Vote against / Oppose     29.986930
Don’t know / Undecided    10.715618
dtype: float64
Raw N: 825
Result 57.0 (80% CI: 47.1 to 66.8) (N=330) (raw_moe=6.7pts, margin=15.0pts, sigma=11.7pts) (72.4% likely to pass)
57.0 (80% CI: 47.1 to 66.8) (72.4%)
-
-
## vote_measure_alt_protein_funding in Virginia ##
Vote for / Support        55.963330
Vote against / Oppose     32.811487
Don’t know / Undecided    11.225183
dtype: float64
Raw N: 825
Result 53.8 (80% CI: 43.7 to 63.8) (N=330) (raw_moe=6.8pts, margin=15.3pts, sigma=11.9pts) (62.4% likely to pass)
53.8 (80% CI: 43.7 to 63.8) (62.4%)
-
-
## vote_measure_alt_protein_funding in Washington ##
Vote for / Support        57.573064
Vote against / Oppose     31.541555
Don’t know / Undecided    10.885380
dtype: f

Result 39.8 (80% CI: 28.3 to 51.4) (N=261) (raw_moe=7.8pts, margin=17.6pts, sigma=13.7pts) (22.9% likely to pass)
39.8 (80% CI: 28.3 to 51.4) (22.9%)
-
-
## vote_measure_utah_hunt in Maine ##
Vote for / Support        41.465889
Vote against / Oppose     43.334106
Don’t know / Undecided    15.200005
dtype: float64
Raw N: 694
Result 40.3 (80% CI: 28.6 to 51.9) (N=271) (raw_moe=7.7pts, margin=17.8pts, sigma=13.9pts) (24.0% likely to pass)
40.3 (80% CI: 28.6 to 51.9) (24.0%)
-
-
## vote_measure_utah_hunt in Maryland ##
Vote for / Support        39.863978
Vote against / Oppose     43.950296
Don’t know / Undecided    16.185726
dtype: float64
Raw N: 694
Result 38.9 (80% CI: 27.0 to 50.8) (N=275) (raw_moe=7.6pts, margin=18.1pts, sigma=14.1pts) (21.6% likely to pass)
38.9 (80% CI: 27.0 to 50.8) (21.6%)
-
-
## vote_measure_utah_hunt in Massachusetts ##
Vote for / Support        40.285574
Vote against / Oppose     43.496423
Don’t know / Undecided    16.218003
dtype: float64
Raw N: 694
Result 39.3

Result 39.8 (80% CI: 28.2 to 51.4) (N=266) (raw_moe=7.8pts, margin=17.6pts, sigma=13.8pts) (23.0% likely to pass)
39.8 (80% CI: 28.2 to 51.4) (23.0%)
-
-
## vote_measure_utah_hunt in Utah ##
Vote for / Support        48.287025
Vote against / Oppose     37.301300
Don’t know / Undecided    14.411675
dtype: float64
Raw N: 694
Result 46.9 (80% CI: 35.3 to 58.5) (N=251) (raw_moe=7.9pts, margin=17.7pts, sigma=13.8pts) (41.1% likely to pass)
46.9 (80% CI: 35.3 to 58.5) (41.1%)
-
-
## vote_measure_utah_hunt in Vermont ##
Vote for / Support        42.202368
Vote against / Oppose     41.622342
Don’t know / Undecided    16.175289
dtype: float64
Raw N: 694
Result 41.2 (80% CI: 29.4 to 53.2) (N=273) (raw_moe=7.6pts, margin=18.1pts, sigma=14.1pts) (26.8% likely to pass)
41.2 (80% CI: 29.4 to 53.2) (26.8%)
-
-
## vote_measure_utah_hunt in Virginia ##
Vote for / Support        40.907728
Vote against / Oppose     43.794560
Don’t know / Undecided    15.297713
dtype: float64
Raw N: 694
Result 39.7 (80% C

Result 67.2 (80% CI: 58.5 to 76.0) (N=272) (raw_moe=6.8pts, margin=13.3pts, sigma=10.4pts) (95.1% likely to pass)
67.2 (80% CI: 58.5 to 76.0) (95.1%)
-
-
## vote_measure_min_wage in Louisiana ##
Vote for / Support        71.539079
Vote against / Oppose     22.485499
Don’t know / Undecided     5.975423
dtype: float64
Raw N: 685
Result 68.0 (80% CI: 59.4 to 76.6) (N=274) (raw_moe=6.7pts, margin=13.1pts, sigma=10.2pts) (96.1% likely to pass)
68.0 (80% CI: 59.4 to 76.6) (96.1%)
-
-
## vote_measure_min_wage in Maine ##
Vote for / Support        71.950182
Vote against / Oppose     20.767795
Don’t know / Undecided     7.282024
dtype: float64
Raw N: 685
Result 68.8 (80% CI: 60.0 to 77.6) (N=281) (raw_moe=6.5pts, margin=13.4pts, sigma=10.5pts) (96.3% likely to pass)
68.8 (80% CI: 60.0 to 77.6) (96.3%)
-
-
## vote_measure_min_wage in Maryland ##
Vote for / Support        73.558911
Vote against / Oppose     19.493468
Don’t know / Undecided     6.947620
dtype: float64
Raw N: 685
Result 70.3 (80% C

Result 67.5 (80% CI: 58.8 to 76.2) (N=273) (raw_moe=6.8pts, margin=13.3pts, sigma=10.4pts) (95.4% likely to pass)
67.5 (80% CI: 58.8 to 76.2) (95.4%)
-
-
## vote_measure_min_wage in Texas ##
Vote for / Support        71.875441
Vote against / Oppose     21.721892
Don’t know / Undecided     6.402667
dtype: float64
Raw N: 685
Result 68.5 (80% CI: 59.8 to 77.1) (N=277) (raw_moe=6.6pts, margin=13.2pts, sigma=10.3pts) (96.4% likely to pass)
68.5 (80% CI: 59.8 to 77.1) (96.4%)
-
-
## vote_measure_min_wage in Utah ##
Vote for / Support        66.258492
Vote against / Oppose     20.375684
Don’t know / Undecided    13.365824
dtype: float64
Raw N: 685
Result 64.6 (80% CI: 54.0 to 75.2) (N=272) (raw_moe=6.8pts, margin=16.1pts, sigma=12.6pts) (87.7% likely to pass)
64.6 (80% CI: 54.0 to 75.2) (87.7%)
-
-
## vote_measure_min_wage in Vermont ##
Vote for / Support        72.014829
Vote against / Oppose     18.899265
Don’t know / Undecided     9.085906
dtype: float64
Raw N: 685
Result 69.3 (80% CI: 60.